# Welcome! #
## Enzyme and Zygote Performance Shootout ##

### Project Information ###

If you're interested in seeing a project synopsis and the benchmarks for performance we'll measuring, please take a look at our project documentation. 

### What Now? ###

Let's start by setting up Enzyme and Zygote in our environment to start testing some differentiation. Our primary benchmark is the physical timing of a differentiation operation. In order to measure this we will simply be using the `@time` Julia method. These times will then be compared to measure how fast differentiation is actually done, along with comparing other important benchmarks.

#### Import the Packages ####

In [1]:
using Pkg

Pkg.add("Enzyme")
Pkg.add("Zygote")

In [2]:
using Zygote
using Enzyme

#### What Next? ####

Now that the packages have been added to the environment, we can start testing them out. First, a quick demonstration of the timing function we will be using. 

In [3]:
function fib(n)
    if n <= 1
        return 1
    else
        return fib(n - 1) + fib(n - 2)
    end
end
    
@time fib(20)

  0.000052 seconds


10946

#### Functions ####

For this shootout we will be handling differentiation for rootfinding problems. Specifically, we will be testing differentiation efficiency on four different rootfinding algorithms, being Halley's, Golbabai-Javidi, Newton's, Noor's, Traub's, and Zhanlav's Method.

Those methods look like such:

**Halley's Method**

$x_{n + 1} = x_{n} - \frac{2f(x_n)f^{\prime}(x_n)}{2f^{\prime}(x_n)^2 - f(x_n)f^{\prime \prime}(x_n)}$

**Golbabai-Javidi Method**

$x_{n + 1} = x_{n} - \frac{f(x_n)}{f^{\prime}(x_n)} - \frac{f(x_n)f^{\prime \prime}(x_n)}{2(f^{\prime \prime \prime}(x_n) - f(x_n)f^{\prime}(x_n)f^{\prime \prime}(x_n))}$

**Newton's Method**

$x_{n + 1} = x_{n} - \frac{f(x_n)}{f^{\prime}(x_n)}$

**Noor's Method**

$y_n = x_n - \frac{f(x_n)}{f^{\prime}(x_n)}$

$x_{n + 1} = x_{n} - \frac{f(x_n)}{f^{\prime}(x_n)} + (\frac{f(x_n)}{f^{\prime}(x_n)})\frac{f^{\prime}(y_n)}{f^{\prime}(x_n)}$

**Zhanlav's Method**

$z_n = y_n - \frac{f(y_n)}{f^{\prime}(y_n)}$

$q_n = z_n - \frac{f(z_n)}{f^{\prime}(y_n)}$

$y_{n + 1} = z_n - \frac{f(z_n) + f(q_n)}{f^{\prime}(y_n)}$

**Usage**

Enzyme autodiff(f, x) returns a touple, so I recommend getting the result with first(autodiff(f, x))

In [45]:

    # testing
    # forward difference method with Enzyme and Zygote's functions.

f(x::Vector) = sum(sin, x) + prod(tan, x) * sum(sqrt, x);
x_ = rand(5)
x2 = rand(5)

g = x -> Enzyme.fwddiff(f, x);
g2 = x2 -> Zygote.forwarddiff(f, x);

println(x)
println(x2)

[0.34061518445275163, 0.7016343449077851, 0.033149278899596624, 0.7954294213464141, 0.1365669352668668]
[0.05316861155705932, 0.8471088880447868, 0.9083088206507779, 0.03460690811222422, 0.3588957183115744]


In [53]:

    # testing Newton's with Enzyme

function newton(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:100 # max number of iterations
        fx = f(x)
        fpx = first(Enzyme.autodiff(f, Active(x)))
        if verbose
            println("[$k] x=$x  f(x)=$fx  f'(x)=$fpx")
        end
        if abs(fx) < tol
            return x, fx, k
        end
        x = x - fx / fpx
    end  
end

f(x) = cos(x) - x
newton(f, 1; tol=1e-15, verbose=true)

[1] x=1  f(x)=-0.45969769413186023  f'(x)=-1.8414709848078965
[2] x=0.7503638678402439  f(x)=-0.018923073822117442  f'(x)=-1.6819049529414878
[3] x=0.7391128909113617  f(x)=-4.6455898990771516e-5  f'(x)=-1.6736325442243012
[4] x=0.739085133385284  f(x)=-2.847205804457076e-10  f'(x)=-1.6736120293089505
[5] x=0.7390851332151607  f(x)=0.0  f'(x)=-1.6736120291832148


(0.7390851332151607, 0.0, 5)